# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 10
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus

In [3]:
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecastPolling
import postOptimization
import numpy as np

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()
    
day_in_minute_interval_steps = list(np.around(np.arange(480, 960 + 1, minute_interval), 1))
distributionAlgorithmForecastPolling.init_simulation(day_in_minute_interval_steps, minute_interval, simulation_data, 
                                                   simulation_day, solarpeakleistung)

postOptimization.start_post_optimization(minute_interval, simulation_day, solarpeakleistung, bev_data, table_dict, simulation_data,
                ladeleistung_pro_bev)

distributionAlgorithmForecastPolling.start_simulation(solarpeakleistung, ladeleistung_pro_bev,
                     simulation_day, bev_data, table_dict, simulation_data, minute_interval)

START POST OPTIMIZATION
charging list per minute dict:  {}


OPTIMIZATION DONE
BEVs dict:  {0: [(8.0, 6.6), 'nicht parkend', [], [33.467499999999994]], 1: [(8.5, 4.6), 'nicht parkend', [], [25.637499999999996]], 2: [(8.0, 7.9), 'nicht parkend', [], [37.21749999999999]], 3: [(12.5, 3.5), 'wartend', [(859.9090909090909, 100.09090909090912, 0)], [14.740000000000002]], 4: [(8.0, 6.0), 'wartend', [(757.7682436611008, 82.23175633889923, 0)], [29.772499999999994]], 5: [(8.5, 3.2), 'nicht parkend', [], [16.537499999999998]], 6: [(8.5, 5.8), 'nicht parkend', [], [31.93499999999999]], 7: [(13.0, 3.0), 'wartend', [(863.3681818181818, 81.63181818181818, 0)], [12.1925]], 8: [(8.0, 7.9), 'nicht parkend', [], [37.21749999999999]], 9: [(9.0, 7.0), 'wartend', [(778.2629841931562, 106.73701580684383, 0)], [35.82749999999999]]}
BEVs from post optimization:  [3, 4, 7, 9]


Minute:  480
charging list per minute dict:  {}
Waiting BEVs:  []
Charging list after handling overload:  []
Waiting BEVs after updati

SET BEV DATA for current minute for ID BEV 1 for minute 600:  10.176
SET BEV DATA for current minute for ID BEV 5 for minute 600:  10.176
Charging BEVs (nachdem neue Plätze belegt):  [0, 2, 8, 1, 5]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 0:  19.119999999999994
Restladezeit:  112.73584905660374
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 2:  24.492499999999986
Restladezeit:  144.41332547169802
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 8:  27.929999999999986
Restladezeit:  164.6816037735848
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 1:  20.849999999999994
Restladezeit:  122.93632075471695
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 5:  14.187499999999998
Restladezeit:  83.65271226415094
Charging BEVs:  [0, 2, 8, 1, 5]


Minute:  615
charging list per minute dict:  {480: [0], 495: [0], 510: [0, 2], 525: [0, 2], 540: [0, 2, 8], 555: [0, 2, 8], 570: [0, 2, 8, 1], 585: [0, 2, 8, 1, 5], 600: [0, 2, 8, 1, 5]}
Waiting

FUEL Charging BEV:  2
FUEL Charging BEV:  8
FUEL Charging BEV:  1
FUEL Charging BEV:  5
SET BEV DATA for current minute for ID BEV 0 for minute 675:  10.376000000000001
SET BEV DATA for current minute for ID BEV 2 for minute 675:  10.376000000000001
SET BEV DATA for current minute for ID BEV 8 for minute 675:  10.376000000000001
SET BEV DATA for current minute for ID BEV 1 for minute 675:  10.376000000000001
SET BEV DATA for current minute for ID BEV 5 for minute 675:  10.376000000000001
Charging BEVs (nachdem neue Plätze belegt):  [0, 2, 8, 1, 5]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 0:  6.488999999999997
Restladezeit:  37.523130300693886
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 2:  11.861499999999985
Restladezeit:  68.59001542020037
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 8:  15.298999999999985
Restladezeit:  88.46761757902843
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 1:  8.218999999999994
Restladezeit:  47.526985

FUEL Charging BEV:  6
SET BEV DATA for current minute for ID BEV 2 for minute 735:  15.96
SET BEV DATA for current minute for ID BEV 8 for minute 735:  15.96
SET BEV DATA for current minute for ID BEV 6 for minute 735:  15.96
Charging BEVs (nachdem neue Plätze belegt):  [2, 8, 6]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 2:  0.35539999999998884
Restladezeit:  1.3360902255638678
Die Restladezeit des BEVS 2 endet vor dem nächsten Interval
SET BEV DATA after charging time over for ID BEV 2 for minute 736.3360902255639:  16.61
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 8:  3.674433333333326
Restladezeit:  13.813659147869645
Die Restladezeit des BEVS 8 endet vor dem nächsten Interval
SET BEV DATA after charging time over for ID BEV 8 for minute 748.8136591478697:  15.293333333333335
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 6:  20.642122222222213
Restladezeit:  77.60196324143688
Charging BEVs:  [2, 8, 6]


Minute:  750
charging list per minute 

charging_list_per_minute_dict  {480: [0], 495: [0], 510: [0, 2], 525: [0, 2], 540: [0, 2, 8], 555: [0, 2, 8], 570: [0, 2, 8, 1], 585: [0, 2, 8, 1, 5], 600: [0, 2, 8, 1, 5], 615: [0, 2, 8, 1, 5], 630: [0, 2, 8, 1, 5], 645: [0, 2, 8, 1, 5], 660: [0, 2, 8, 1, 5], 675: [0, 2, 8, 1, 5], 690: [0, 2, 8, 1, 6], 705: [0, 2, 8, 1, 6], 720: [2, 8, 1, 6], 735: [2, 8, 6], 750: [6, 4], 765: [6, 4, 9], 780: [6, 4, 9], 795: [6, 4, 9], 810: [6, 4, 9]}
FUEL Charging BEV:  4
FUEL Charging BEV:  9
SET BEV DATA for current minute for ID BEV 4 for minute 825:  20.625
SET BEV DATA for current minute for ID BEV 9 for minute 825:  20.625
Charging BEVs (nachdem neue Plätze belegt):  [4, 9]
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 4:  13.808459259259244
Restladezeit:  40.170063299663255
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 9:  25.290033333333334
Restladezeit:  73.57100606060607
Charging BEVs:  [4, 9]


Minute:  840
charging list per minute dict:  {480: [0], 495: [0], 510: [0

Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 3:  4.952499999999995
Restladezeit:  48.00484652665585
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 7:  3.0391666666666683
Restladezeit:  29.45880452342489
Charging BEVs:  [3, 7]


Minute:  945
charging list per minute dict:  {480: [0], 495: [0], 510: [0, 2], 525: [0, 2], 540: [0, 2, 8], 555: [0, 2, 8], 570: [0, 2, 8, 1], 585: [0, 2, 8, 1, 5], 600: [0, 2, 8, 1, 5], 615: [0, 2, 8, 1, 5], 630: [0, 2, 8, 1, 5], 645: [0, 2, 8, 1, 5], 660: [0, 2, 8, 1, 5], 675: [0, 2, 8, 1, 5], 690: [0, 2, 8, 1, 6], 705: [0, 2, 8, 1, 6], 720: [2, 8, 1, 6], 735: [2, 8, 6], 750: [6, 4], 765: [6, 4, 9], 780: [6, 4, 9], 795: [6, 4, 9], 810: [6, 4, 9], 825: [4, 9], 840: [9], 855: [9, 3, 7], 870: [9, 3, 7], 885: [9, 3, 7], 900: [3, 7], 915: [3, 7], 930: [3, 7]}
Waiting BEVs:  []
SET BEV DATA after charging time over for ID BEV 3 for minute 945:  5.625
Removed from charging list because of overload:  3
Charging list after handling overload:  [7

### Ergebnis Algorithmus

In [4]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag, solarpeakleistung, minute_interval)

>**Ergebnis**<br>Geladene BEVs: 10 von 10<br>Verfügbare Solarenergie insgesamt: 276.47 kWh<br>Aufgeladene Solarenergie insgesamt: 255.31 kWh<br>Ungenutzte Solarenergie insgesamt: 21.16 kWh

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [5]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict, minute_interval)

interactive(children=(IntSlider(value=480, description='Minute: ', max=960, min=480, step=15), Output()), _dom…

In [6]:
import figureGeneration
# NICHT MEHR RICHTIG, da BEVs aus post optimization aktuell nicht auf der Warteliste landen
# figureGeneration.create_bev_number_figure(simulation_data)

In [7]:
figureGeneration.create_charging_power_figure(simulation_day, solarpeakleistung, bev_data, minute_interval)